## Installing packages

In [1]:
# Linking to drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
# Loading the drive
import os
os.chdir("/content/gdrive/Shareddrives/Low_Budget_MT")

In [3]:
#! pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install . --use-feature=in-tree-build

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt
     |████████████████████████████████| 116 kB 27.9 MB/s 
     |████████████████████████████████| 507 kB 58.3 MB/s 
     |████████████████████████████████| 264 kB 70.3 MB/s 
     |████████████████████████████████| 103 kB 70.8 MB/s 
     |████████████████████████████████| 843 kB 46.6 MB/s 
     |████████████████████████████████| 57 kB 6.1 MB/s 
  Created wheel for joeynmt: filename=joeynmt-1.5.1-py3-none-any.whl size=86509 sha256=8ea06b2c4078c35b462fe48bcec5ebc20ccd146f34fff75c6aa0f85ce2bfdf7d
  Stored in directory: /tmp/pip-ephem-wheel-cache-bd1nrk_f/wheels/ec/b0/4e/e15e388dd23d01f1c2b4065e1bb36099e5e22abe4f7147f706
  Created wheel for wrapt: filename=wrapt-1.11.1-cp37-cp37m-linux_x86_64.whl size=68435 sha256=f184eec454cfb57d27ccd96ede528b95c134d093bb57342a2769d4301801e5ed
  Stored in directory: /root/.cache/pip/wheels/4e/58/9d

In [4]:
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 33.3 MB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [5]:
# Importing needed libraries for preprocessing and visualization
from my_helpers import rttl_data_load, split_srctgt, generating_BPE, run_model, rttl_scoring, nrttl_data_load

import warnings
warnings.filterwarnings('ignore')

# K1

## Iteration 2

In [ ]:
rttl_scoring('/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k1/nrttl/nrttl_rem.csv', 'en', 'rw','nrttl', 'k1', '2','transformer_reverse_enrw2_nrttl','transformer_enrw2_nrttl_reload')

(95000, 2)
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k1/nrttl/2
/content/gdrive/Shareddrives/Low_Budget_MT
Starting here
95000 95000 95000
95000
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k1/nrttl/2
/content/gdrive/Shareddrives/Low_Budget_MT
doing scoring


# K2

## Iteration 1

### Scoring

In [ ]:
!cp -a /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k2/random/1/. /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k2/nrttl/1/

In [ ]:
def rttl_scoring(rem_file, src, tgt, query, k, iteration, M, Mrev):

  path = '/content/gdrive/Shareddrives/Low_Budget_MT/data/'+tgt+'/'+k+'/'+query+'/'+iteration
  df = pd.read_csv(rem_file)
  print(df.shape)

  os.chdir(path)
  print(os.getcwd())

  # Splitting x and y:
  os.environ['src'] = src
  os.environ['tgt']  = tgt
  with open("rem."+src, "w") as src_file, open("rem."+tgt, "w") as trg_file:
    for index, row in df.iterrows():
      src_file.write(str(row["source_sentence"])+"\n")
      trg_file.write(str(row["target_sentence"])+"\n")

  # Getting bpe for rem
  #os.system(' subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < rem.$tgt > rem.bpe.$tgt')
  os.chdir('/content/gdrive/Shareddrives/Low_Budget_MT')
  print(os.getcwd())

  #translate rem
  os.environ["model_path"] = '/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/'+k+'/'+M+'/config.yaml'
  os.environ["x_path"] = path+'/rem.bpe.'+tgt
  os.environ["yhat_path"] = path+'/yhat.'+src

  #os.system('cd joeynmt; python -m joeynmt translate $model_path < $x_path > $yhat_path')

  print('Starting here')
  with open(path+'/rem.'+src, "r") as x_file, open(path+'/rem.bpe.'+tgt, "r") as y_file, open(path+'/yhat.'+src, "r") as z_file:
    x = x_file.read().splitlines()
    y = y_file.read().splitlines()
    z = z_file.read().splitlines()
    print(len(x),len(y),len(z))

  with open(path+'/rem.'+tgt, "r") as a_file:
    a = a_file.read().splitlines()

    print(len(a))

  assert len(x)==len(y)==len(z), "Error in translating"

  f = [i for i,j in enumerate(z) if len(j)==0]

  if len(f) !=0:
    x = [i for j, i in enumerate(x) if j not in f]
    y = [i for j, i in enumerate(y) if j not in f]
    z = [i for j, i in enumerate(z) if j not in f]
    a = [i for j, i in enumerate(a) if j not in f]
    
    print(len(x),len(y),len(z))
  
  with open(path+'/rem.'+src, "w") as x_file, open(path+'/rem.bpe.'+tgt, "w") as y_file, open(path+'/yhat.'+src, "w") as z_file:
    for i in zip(x,y,z):
        x_file.write(i[0]+"\n")
        y_file.write(i[1]+"\n")
        z_file.write(i[2]+"\n")

  with open(path+'/rem.'+tgt, "w") as a_file:
    for i in a:
        a_file.write(i+"\n")

  os.chdir(path)
  print(os.getcwd())
  # Getting bpe for rem
  os.system(' subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < yhat.$src > rem.bpe.$src')
  os.chdir('/content/gdrive/Shareddrives/Low_Budget_MT')
  print(os.getcwd())

  if os.path.exists(path+'/rem.bpe.'+src):
    config = run_model(tgt,src,tgt,Mrev,query,k,iteration)
    reload_config = config.replace(
        f'test:  "{path}/test.bpe"', f'test:  "{path}/rem.bpe"').replace(
            f'score_mode: test', f'score_mode: scoring').replace(
            f'beam_size: 5', f'beam_size: 1')
            
    with open("joeynmt/models/"+k+"/"+Mrev+"/config.yaml",'w') as f:
        f.write(reload_config)

    os.environ["model_path"] = '/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/'+k+'/'+Mrev+'/config.yaml'
    os.environ["output_path"] = path+'/translation'
    print('doing scoring')
    os.system('cd joeynmt; python -m joeynmt test $model_path --output_path $output_path')
  else:
    print('BPE file not created.')

  with open(path+'/translation.test', "r") as x_file, open(path+'/translation.test.scores', "r") as y_file:
    x = x_file.read().splitlines()
    y = y_file.read().splitlines()
    print(len(x),len(y))

In [ ]:
rttl_scoring('/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k2/rem.csv', 'en', 'rw','nrttl', 'k2', '1','transformer_reverse_enrw1','transformer_enrw1')

(100000, 2)
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k2/nrttl/1
/content/gdrive/Shareddrives/Low_Budget_MT
Starting here
100000 100000 100000
100000
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k2/nrttl/1
/content/gdrive/Shareddrives/Low_Budget_MT
doing scoring
100000 100000


# K3

## Iteration 1

### Scoring

In [ ]:
!cp -a /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k3/random/1/. /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k3/nrttl/1/

In [ ]:
rttl_scoring('/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k3/rem.csv', 'en', 'rw','nrttl', 'k3', '1','transformer_reverse_enrw1','transformer_enrw1')

(100000, 2)
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k3/nrttl/1
/content/gdrive/Shareddrives/Low_Budget_MT
Starting here
100000 100000 100000
100000
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k3/nrttl/1
/content/gdrive/Shareddrives/Low_Budget_MT
doing scoring
100000 100000


# K4

## Iteration 1

### Scoring

In [ ]:
!cp -a /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k4/random/1/. /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k4/nrttl/1/

In [ ]:
rttl_scoring('/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k4/rem.csv', 'en', 'rw','nrttl', 'k4', '1','transformer_reverse_enrw1','transformer_enrw1')

(100000, 2)
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k4/nrttl/1
/content/gdrive/Shareddrives/Low_Budget_MT
Starting here
100000 100000 100000
100000
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k4/nrttl/1
/content/gdrive/Shareddrives/Low_Budget_MT
doing scoring
100000 100000


# K5

## Iteration 1

### Scoring

In [ ]:
!cp -a /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k5/random/1/. /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k5/nrttl/1/

In [ ]:
rttl_scoring('/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k5/rem.csv', 'en', 'rw','nrttl', 'k5', '1','transformer_reverse_enrw1','transformer_enrw1')

(100000, 2)
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k5/nrttl/1
/content/gdrive/Shareddrives/Low_Budget_MT
Starting here
100000 100000 100000
100000
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k5/nrttl/1
/content/gdrive/Shareddrives/Low_Budget_MT
doing scoring
100000 100000


In [ ]:
print('i got here')